In [ ]:
# Run this cell only when you are troubled with the following error especially if you are using conda to manage python environment
# `ImportError: libpython3.8.so.1.0: cannot open shared object file: No such file or directory
import os
from pathlib import Path
from ctypes import cdll
import sys

is_conda = 'CONDA_PREFIX' in os.environ or 'CONDA_DEFAULT_ENV' in os.environ
if is_conda:
    version_info = sys.version_info
    if version_info.major == 3 and version_info.minor >= 8:
        conda_lib_path = Path(
            sys.executable).parent.parent / f"lib/libpython{version_info.major}.{version_info.minor}.so.1.0"
    else:
        conda_lib_path = Path(
            sys.executable).parent.parent / f"lib/libpython{version_info.major}.{version_info.minor}m.so.1.0"
    python_lib = cdll.LoadLibrary(str(conda_lib_path))
    print(f"Load Python lib {conda_lib_path}")


In [ ]:
import os
from datetime import datetime
import numpy as np

# noinspection PyUnresolvedReferences
import isaacgym
from sim_web_visualizer.isaac_visualizer_client import create_isaac_visualizer, bind_visualizer_to_gym, set_gpu_pipeline
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess

sys.path.append(os.path.join(os.path.abspath(''), 'IsaacGymEnvs'))

import hydra

from isaacgymenvs.pbt.pbt import PbtAlgoObserver, initial_pbt_check
from hydra.utils import to_absolute_path
from isaacgymenvs.tasks import isaacgym_task_map
from omegaconf import DictConfig, OmegaConf
import gym

from isaacgymenvs.utils.reformat import omegaconf_to_dict, print_dict
from isaacgymenvs.utils.utils import set_np_formatting, set_seed

In [ ]:
# This is equivalent to run `meshcat-server` inside a separate terminal window
start_zmq_server_as_subprocess()

################################################################################################
# The following code is only used for visualizer and not presented in the original IsaacGymEnv
################################################################################################
from isaacgymenvs.tasks.base import vec_task
from isaacgym import gymapi


def wrapped_create_sim(self: vec_task.VecTask, compute_device: int, graphics_device: int, physics_engine,
                       sim_params: gymapi.SimParams):
    sim = vec_task._create_sim_once(self.gym, compute_device, graphics_device, physics_engine, sim_params)
    if sim is None:
        print("*** Failed to create sim")
        quit()
    self.gym = bind_visualizer_to_gym(self.gym, sim)
    set_gpu_pipeline(sim_params.use_gpu_pipeline)
    return sim


# Reload VecTask function to create a hook for sim_web_visualizer
vec_task.VecTask.create_sim = wrapped_create_sim

# Create web visualizer
web_visualizer = create_isaac_visualizer(port=6000, host="localhost", keep_default_viewer=False, max_env=16,
                                         scene_offset=np.array([40.0, 40.0]))


################################################################################################
# End of visualizer code
################################################################################################


In [ ]:
################################################################################################
# The following code is the same as the original IsaacGymEnv
# https://github.com/NVIDIA-Omniverse/IsaacGymEnvs/blob/main/isaacgymenvs/train.py
################################################################################################

def preprocess_train_config(cfg, config_dict):
    """
    Adding common configuration parameters to the rl_games train config.
    An alternative to this is inferring them in task-specific .yaml files, but that requires repeating the same
    variable interpolations in each config.
    """

    train_cfg = config_dict['params']['config']

    train_cfg['device'] = cfg.rl_device

    train_cfg['population_based_training'] = cfg.pbt.enabled
    train_cfg['pbt_idx'] = cfg.pbt.policy_idx if cfg.pbt.enabled else None

    train_cfg['full_experiment_name'] = cfg.get('full_experiment_name')

    print(f'Using rl_device: {cfg.rl_device}')
    print(f'Using sim_device: {cfg.sim_device}')
    print(train_cfg)

    try:
        model_size_multiplier = config_dict['params']['network']['mlp']['model_size_multiplier']
        if model_size_multiplier != 1:
            units = config_dict['params']['network']['mlp']['units']
            for i, u in enumerate(units):
                units[i] = u * model_size_multiplier
            print(
                f'Modified MLP units by x{model_size_multiplier} to {config_dict["params"]["network"]["mlp"]["units"]}')
    except KeyError:
        pass

    return config_dict


def launch_rlg_hydra(cfg: DictConfig):
    if cfg.pbt.enabled:
        initial_pbt_check(cfg)

    from isaacgymenvs.utils.rlgames_utils import RLGPUEnv, RLGPUAlgoObserver, MultiObserver, ComplexObsRLGPUEnv
    from isaacgymenvs.utils.wandb_utils import WandbAlgoObserver
    from rl_games.common import env_configurations, vecenv
    from rl_games.torch_runner import Runner
    from rl_games.algos_torch import model_builder
    from isaacgymenvs.learning import amp_continuous
    from isaacgymenvs.learning import amp_players
    from isaacgymenvs.learning import amp_models
    from isaacgymenvs.learning import amp_network_builder
    import isaacgymenvs

    time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_name = f"{cfg.wandb_name}_{time_str}"

    # ensure checkpoints can be specified as relative paths
    if cfg.checkpoint:
        cfg.checkpoint = to_absolute_path(cfg.checkpoint)

    cfg_dict = omegaconf_to_dict(cfg)
    print_dict(cfg_dict)

    # set numpy formatting for printing only
    set_np_formatting()

    # global rank of the GPU
    global_rank = int(os.getenv("RANK", "0"))

    # sets seed. if seed is -1 will pick a random one
    cfg.seed = set_seed(cfg.seed, torch_deterministic=cfg.torch_deterministic, rank=global_rank)

    def create_isaacgym_env(**kwargs):
        envs = isaacgymenvs.make(
            cfg.seed,
            cfg.task_name,
            cfg.task.env.numEnvs,
            cfg.sim_device,
            cfg.rl_device,
            cfg.graphics_device_id,
            cfg.headless,
            cfg.multi_gpu,
            cfg.capture_video,
            cfg.force_render,
            cfg,
            **kwargs,
        )
        if cfg.capture_video:
            envs.is_vector_env = True
            envs = gym.wrappers.RecordVideo(
                envs,
                f"videos/{run_name}",
                step_trigger=lambda step: step % cfg.capture_video_freq == 0,
                video_length=cfg.capture_video_len,
            )
        return envs

    env_configurations.register('rlgpu', {
        'vecenv_type': 'RLGPU',
        'env_creator': lambda **kwargs: create_isaacgym_env(**kwargs),
    })

    ige_env_cls = isaacgym_task_map[cfg.task_name]
    dict_cls = ige_env_cls.dict_obs_cls if hasattr(ige_env_cls, 'dict_obs_cls') and ige_env_cls.dict_obs_cls else False

    if dict_cls:

        obs_spec = {}
        actor_net_cfg = cfg.train.params.network
        obs_spec['obs'] = {'names': list(actor_net_cfg.inputs.keys()),
                           'concat': not actor_net_cfg.name == "complex_net", 'space_name': 'observation_space'}
        if "central_value_config" in cfg.train.params.config:
            critic_net_cfg = cfg.train.params.config.central_value_config.network
            obs_spec['states'] = {'names': list(critic_net_cfg.inputs.keys()),
                                  'concat': not critic_net_cfg.name == "complex_net", 'space_name': 'state_space'}

        vecenv.register('RLGPU',
                        lambda config_name, num_actors, **kwargs: ComplexObsRLGPUEnv(config_name, num_actors, obs_spec,
                                                                                     **kwargs))
    else:

        vecenv.register('RLGPU', lambda config_name, num_actors, **kwargs: RLGPUEnv(config_name, num_actors, **kwargs))

    rlg_config_dict = omegaconf_to_dict(cfg.train)
    rlg_config_dict = preprocess_train_config(cfg, rlg_config_dict)

    observers = [RLGPUAlgoObserver()]

    if cfg.pbt.enabled:
        pbt_observer = PbtAlgoObserver(cfg)
        observers.append(pbt_observer)

    if cfg.wandb_activate:
        cfg.seed += global_rank
        if global_rank == 0:
            # initialize wandb only once per multi-gpu run
            wandb_observer = WandbAlgoObserver(cfg)
            observers.append(wandb_observer)

    # register new AMP network builder and agent
    def build_runner(algo_observer):
        runner = Runner(algo_observer)
        runner.algo_factory.register_builder('amp_continuous', lambda **kwargs: amp_continuous.AMPAgent(**kwargs))
        runner.player_factory.register_builder('amp_continuous',
                                               lambda **kwargs: amp_players.AMPPlayerContinuous(**kwargs))
        model_builder.register_model('continuous_amp', lambda network, **kwargs: amp_models.ModelAMPContinuous(network))
        model_builder.register_network('amp', lambda **kwargs: amp_network_builder.AMPBuilder())

        return runner

    # convert CLI arguments into dictionary
    # create runner and set the settings
    runner = build_runner(MultiObserver(observers))
    runner.load(rlg_config_dict)
    runner.reset()

    # dump config dict
    if not cfg.test:
        experiment_dir = os.path.join('runs', cfg.train.params.config.name +
                                      '_{date:%d-%H-%M-%S}'.format(date=datetime.now()))

        os.makedirs(experiment_dir, exist_ok=True)
        with open(os.path.join(experiment_dir, 'config.yaml'), 'w') as f:
            f.write(OmegaConf.to_yaml(cfg))

    runner.run({
        'train': not cfg.test,
        'play': cfg.test,
        'checkpoint': cfg.checkpoint,
        'sigma': cfg.sigma if cfg.sigma != '' else None
    })

In [ ]:
# Comment out the hydra/job_logging in the config to avoid a hydra error.
# This is nothing related to the web visualizer but will raise a multiple value error without the following modification
# You do not need to do that if you are not using hydra compose API.
!sed -i 's/- hydra/#- hydra/g' IsaacGymEnvs/isaacgymenvs/cfg/config.yaml

In [ ]:
# Modify the IP if the jupyter server is not localhost
# If your remote server has a public ip, the machine ip should be that one
# If you are using ssh port forwarding like: ssh YOUR_SERVER -L 7000:localhost:7000 to forward port 7000 from your server to local machine, it should still be your localhost
machine_ip = "127.0.0.1"
web_visualizer.jupyter_cell(url=f"http://{machine_ip}:7000/static/", height=400)

In [ ]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
config_path = "IsaacGymEnvs/isaacgymenvs/cfg"
task_name = "Ant"
with hydra.initialize(version_base=None, config_path=config_path):
    cfg = hydra.compose(config_name='config.yaml', overrides=[f"task={task_name}"])
launch_rlg_hydra(cfg)